In [12]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import numpy as np
import base64
from collections import defaultdict

def ecb(plain, key):
    aes = Cipher(algorithms.AES128(key), modes.ECB()).encryptor()
    pad = padding.PKCS7(128).padder()
    return aes.update(pad.update(plain) + pad.finalize()) + aes.finalize()

def pkcs7(b, size=128):
    bsz = len(b)
    size //= 8
    sz = size * (bsz // size + 1)
    return b + bytes([sz - bsz]) * (sz - bsz)

def uses_ecb(cipher):
    counts = defaultdict(int)

    encrypted = cipher(b'A' * 52)

    for blk in np.frombuffer(encrypted, 'u1').reshape(-1, 16):
        counts[blk.tobytes()] += 1
    
    return np.any(np.array(list(counts.values())) > 1)

def cipher_blksz(cipher):
    p = b""
    e = cipher(p)
    s = len(e)
    offset = 0
    while s == len(e):
        p += b'A'
        e = cipher(p)
        offset += 1
    return len(e) - s, offset

def sb_ecb_break(cipher):
    """Single-Byte (Byte-at-a-time) ECB decryption (Simple: no prefix)"""

    blksz, _ = cipher_blksz(cipher)

    if not uses_ecb(cipher):
        return -1

    tgtsz = len(cipher(b""))

    plain = b'A' * blksz

    for i in range(tgtsz):
        crafted = b'A' * (blksz - (i % blksz) - 1)
        crafted = cipher(crafted)
        crafted = [crafted[i:i+blksz] for i in range(0, len(crafted), blksz)]

        brute = [cipher(plain[-(blksz - 1):] + bytes([i])) for i in range(128)]
        brute = [[b[i:i+blksz] for i in range(0, len(b), blksz)] for b in brute]

        for j, b in enumerate(brute):
            if crafted[i // blksz] == b[0]:
                plain += bytes([j])
                break
    
    return plain[16:]

def get_prefix_size(cipher):
    blksz, _ = cipher_blksz(cipher)

    ini = cipher(b"")
    ini_blks = [ini[i:i+blksz] for i in range(0, len(ini), blksz)]
    p = b"A"
    e = cipher(p)
    e_blks = [e[i:i+blksz] for i in range(0, len(e), blksz)]

    # get initial potential full blocks
    i = 0
    while ini_blks[i] == e_blks[i]:
        i += 1
    psz = i * blksz

    pblk = ini_blks[i]
    nblk = e_blks[i]

    while pblk != nblk:
        p += b"A"
        e = cipher(p)
        e_blks = [e[i:i+blksz] for i in range(0, len(e), blksz)]
        pblk = nblk
        nblk = e_blks[i]
    
    return psz + blksz - len(p) + 1

In [3]:
K = np.random.bytes(16)
P = np.random.bytes(np.random.randint(5, 40))
S = b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'


In [4]:
def encryption_oracle(plain, key=K, secret=S):
    plain = pkcs7(P + plain + base64.decodebytes(secret))
    return ecb(plain, key)

In [5]:
def forged_oracle(oracle):
    blksz, _ = cipher_blksz(oracle)
    psz = get_prefix_size(oracle)
    ppad = blksz - psz % blksz
    return lambda plain: oracle(b'A' * ppad + plain)[(psz // blksz + 1) * blksz:]

In [6]:
print(sb_ecb_break(forged_oracle(encryption_oracle)).decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by
?????????????????????


In [7]:
def sanitize(s):
    return s.replace(b'&', b'').replace(b'=', b'')

def sanitize1(s):
    return s.replace(b'&', b'')

def sanitize2(s):
    return s.replace(b'=', b'')

In [8]:
def profile_oracle(email):
    return ecb(b'email=' + sanitize(email) + b'&uid=10&role=user', K)

def profile_oracle1(email):
    return ecb(b'email=' + sanitize1(email) + b'&uid=10&role=user', K)

def profile_oracle2(email):
    return ecb(b'email=' + sanitize2(email) + b'&uid=10&role=user', K)

In [13]:
sb_ecb_break(forged_oracle(profile_oracle))

b'&&&&&&&&&&&&&&&&'

In [14]:
sb_ecb_break(forged_oracle(profile_oracle1))

b'&&&&&&&&&&&&&&&&'

In [15]:
sb_ecb_break(forged_oracle(profile_oracle2))

b'&uid'

sanitazation makes it so we can't feed some bytes through the oracle which breaks this hack